In [ ]:
!pip install -U pm4py;

In [ ]:
from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import gzip
import io
import pandas as pd
import re

In [ ]:
import pm4py

In [ ]:
file2 = '/content/event-log.xes.gz'
file = '/content/BPI_Challenge_2012.xes.gz'
testfile = '/content/running-example.xes'

In [ ]:
log = pm4py.read_xes(file)
df = pm4py.convert_to_dataframe(log)
# df

parsing log, completed traces ::   0%|          | 0/13087 [00:00<?, ?it/s]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [ ]:
df2 = df.copy()

In [ ]:
dictname = {'A_ACCEPTED': 0, 'A_ACTIVATED': 1, 'A_APPROVED': 2, 'A_CANCELLED': 3, 'A_DECLINED': 4, 'A_FINALIZED': 5, 'A_PARTLYSUBMITTED': 6, 'A_PREACCEPTED': 7, 'A_REGISTERED': 8,'A_SUBMITTED': 9, 'O_ACCEPTED': 10, 'O_CANCELLED': 11, 'O_CREATED': 12, 'O_DECLINED': 13, 'O_SELECTED': 14, 'O_SENT': 15, 'O_SENT_BACK': 16, 'W_Afhandelen leads': 17, 'W_Beoordelen fraude': 18, 'W_Completeren aanvraag': 19,'W_Nabellen incomplete dossiers': 20, 'W_Nabellen offertes': 21, 'W_Valideren aanvraag': 22, 'W_Wijzigen contractgegevens': 23}

In [ ]:
df2 = df2.fillna(0)
df2 = df2.replace({'COMPLETE': 2, 'SCHEDULE': 1, 'START': 0})
df2 = df2.replace(dictname)

In [ ]:
df2

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ
0,112,2,9,2011-10-01 00:38:44.546000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
1,112,2,6,2011-10-01 00:38:44.880000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
2,112,2,7,2011-10-01 00:39:37.906000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
3,112,1,19,2011-10-01 00:39:38.875000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
4,0,0,19,2011-10-01 11:36:46.437000+02:00,2011-10-01 00:38:44.546000+02:00,173688,20000
...,...,...,...,...,...,...,...
262195,112,2,6,2012-02-29 23:51:17.423000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000
262196,112,1,17,2012-02-29 23:52:01.287000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000
262197,11169,0,17,2012-03-01 09:26:46.736000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000
262198,11169,2,4,2012-03-01 09:27:37.118000+01:00,2012-02-29 23:51:16.799000+01:00,214376,15000


In [ ]:
cases = list(set(df2['case:concept:name']))
train_cases = cases[:int(0.75*len(cases))]
test_cases = cases[int(0.75*len(cases)):]
len(cases), len(train_cases)+ len(test_cases), len(train_cases), len(test_cases), train_cases[-1], test_cases[0]

(13087, 13087, 9815, 3272, '177938', '191797')

In [ ]:
df3 = df2.copy()
# df3 = df3.drop('case:REG_DATE', axis = 1)

In [ ]:
df3['time:timestamp'] = pd.to_datetime(df3['time:timestamp'], utc = True).astype(int) #/ 10**9
df3['case:REG_DATE'] = pd.to_datetime(df3['case:REG_DATE'], utc = True).astype(int) #/ 10**9

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: casting datetime64[ns, UTC] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: casting datetime64[ns, UTC] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  


In [ ]:
float(df3[:1]['time:timestamp'])

1317422324.5459998

In [ ]:
df_test, df_train = [x for _, x in df3.groupby(df3['case:concept:name'].isin(train_cases))]

In [ ]:
len(list(set(df_test['case:concept:name']))), len(list(set(df_train['case:concept:name'])))

(3272, 9815)

In [ ]:
train_labels = np.array(df_train['time:timestamp'])
test_labels = np.array(df_test['time:timestamp'])

In [ ]:
train_features = df_train.drop('time:timestamp', axis = 1)
test_features = df_test.drop('time:timestamp', axis = 1)

In [ ]:
train_features = np.array(train_features)
test_features = np.array(test_features)

In [ ]:
# feature_list = list(features.columns)
# features = np.array(features)

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (197954, 5)
Training Labels Shape: (197954,)
Testing Features Shape: (64246, 5)
Testing Labels Shape: (64246,)


In [ ]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 375147.17 degrees.


In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 99.97 %.


In [ ]:
predictions, test_labels

(array([1.31745195e+09, 1.31745194e+09, 1.31745194e+09, ...,
        1.33096486e+09, 1.33123698e+09, 1.33125211e+09]),
 array([1.31744947e+09, 1.31744947e+09, 1.31744951e+09, ...,
        1.33062976e+09, 1.33137998e+09, 1.33138025e+09]))

In [ ]:
test_features

array([['112', 2, 9, '173697', '15000'],
       ['112', 2, 6, '173697', '15000'],
       ['112', 2, 4, '173697', '15000'],
       ...,
       ['10933', 2, 19, '214373', '8500'],
       ['11119', 0, 21, '214373', '8500'],
       ['11119', 2, 21, '214373', '8500']], dtype=object)

In [ ]:
train_labels2 = np.array(df_train[['lifecycle:transition',	'concept:name']])
test_labels2 = np.array(df_test[['lifecycle:transition',	'concept:name']])

In [ ]:
train_features2 = df_train.drop(['lifecycle:transition',	'concept:name'], axis = 1)
test_features2 = df_test.drop(['lifecycle:transition',	'concept:name'], axis = 1)

In [ ]:
train_features2 = np.array(train_features2)
test_features2 = np.array(test_features2)

In [ ]:
print('Training Features Shape:', train_features2.shape)
print('Training Labels Shape:', train_labels2.shape)
print('Testing Features Shape:', test_features2.shape)
print('Testing Labels Shape:', test_labels2.shape)

Training Features Shape: (197954, 5)
Training Labels Shape: (197954, 2)
Testing Features Shape: (64246, 5)
Testing Labels Shape: (64246, 2)


In [ ]:
# Instantiate model with 1000 decision trees
rf2 = RandomForestRegressor(n_estimators = 100, random_state = 42)
# Train the model on training data
rf2.fit(train_features2, train_labels2);

In [ ]:
# Use the forest's predict method on the test data
predictions2 = rf2.predict(test_features2)
# Calculate the absolute errors
errors2 = abs(predictions2 - test_labels2)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 375147.17 degrees.


In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape2 = 100 * (errors2[0] / test_labels2[0])
mape3 = 100 * (errors2[1] / test_labels2[1])
# Calculate and display accuracy
accuracy2 = 100 - np.mean(mape2)
accuracy3 = 100 - np.mean(mape3)
print('Accuracy:', round(accuracy2, 2), '%.')
print('Accuracy:', round(accuracy3, 2), '%.')

Accuracy: 92.69 %.
Accuracy: 66.37 %.


In [ ]:
predictions2, test_labels2

(array([[ 1.81315079,  9.47467857],
        [ 1.81315079,  9.47467857],
        [ 1.81315079,  9.47467857],
        ...,
        [ 1.51022619, 17.41867857],
        [ 1.52441667, 20.47841667],
        [ 1.52441667, 20.47841667]]), array([[ 2,  9],
        [ 2,  6],
        [ 2,  4],
        ...,
        [ 2, 19],
        [ 0, 21],
        [ 2, 21]]))

In [ ]:
df_show = pd.DataFrame()
df_show = pd.concat([df_show, pd.DataFrame(test_features)], axis=1)
df_show = pd.concat([df_show, pd.DataFrame(test_labels)], axis=1)
df_show = pd.concat([df_show, pd.DataFrame(predictions)], axis=1)
df_show = pd.concat([df_show, pd.DataFrame(predictions2)], axis=1)

In [ ]:
df_show.columns = ['org:resource', 'lifecycle:transition', 'concept:name',
       'case:concept:name', 'case:AMOUNT_REQ', 'time:timestamp', 'prediction_timestamp', 'pred_lifecycle', 'pred_concept:name']

In [ ]:
df_show['time:timestamp'] = pd.to_datetime(df_show['time:timestamp']* 10**9, utc = True)
df_show['prediction_timestamp'] = pd.to_datetime(df_show['prediction_timestamp']* 10**9, utc = True)

In [ ]:
df_show['difference'] = df_show['prediction_timestamp'] - df_show['time:timestamp']

In [ ]:
df_show

,org:resource,lifecycle:transition,concept:name,case:concept:name,case:AMOUNT_REQ,time:timestamp,prediction_timestamp,pred_lifecycle,pred_concept:name,difference
0,112,2,9,173697,15000,2011-10-01 06:11:08.865999872+00:00,2011-10-01 06:52:25.813901056+00:00,1.813151,9.474679,0 days 00:41:16.947901184
1,112,2,6,173697,15000,2011-10-01 06:11:09.035000064+00:00,2011-10-01 06:52:22.713580288+00:00,1.813151,9.474679,0 days 00:41:13.678580224
2,112,2,4,173697,15000,2011-10-01 06:11:46.420000+00:00,2011-10-01 06:52:22.713580288+00:00,1.813151,9.474679,0 days 00:40:36.293580288
3,112,2,9,173700,5000,2011-10-01 06:15:39.894000128+00:00,2011-10-01 06:12:59.852052224+00:00,1.768067,10.138437,-1 days +23:57:19.958052096
4,112,2,6,173700,5000,2011-10-01 06:15:40.102000128+00:00,2011-10-01 06:13:00.939034624+00:00,1.768067,10.138437,-1 days +23:57:20.837034496
...,...,...,...,...,...,...,...,...,...,...
64241,10933,2,15,214373,8500,2012-03-01 19:22:40.043000064+00:00,2012-03-05 20:03:25.746461184+00:00,1.510226,17.418679,4 days 00:40:45.703461120
64242,10933,1,21,214373,8500,2012-03-01 19:22:40.148999936+00:00,2012-03-03 15:39:48.152761344+00:00,1.510226,17.418679,1 days 20:17:08.003761408
64243,10933,2,19,214373,8500,2012-03-01 19:22:41.156999936+00:00,2012-03-05 16:27:42.340993536+00:00,1.510226,17.418679,3 days 21:05:01.183993600
64244,11119,0,21,214373,8500,2012-03-10 11:46:22.700000+00:00,2012-03-08 20:03:04.257765376+00:00,1.524417,20.478417,-2 days +08:16:41.557765376
